In [1]:
import codecs 
import glob 
import multiprocessing 
import os 
import re 
import nltk 
import gensim.models.word2vec as w2v 

D:\Programs\Anaconda\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#Все книги
book_filenames = sorted(glob.glob("data/*.txt"))
book_filenames

['data\\got1.txt',
 'data\\got2.txt',
 'data\\got3.txt',
 'data\\got4.txt',
 'data\\got5.txt']

In [3]:
#Собираем все книги в одну большую длиннющую строку
corpus_raw = ""
for book_filename in book_filenames:
    print('Считывается %s' % book_filename)
    with codecs.open(book_filename,'r','utf-8') as file:
        corpus_raw += file.read()
    print('Сборка длиной %d' % len(corpus_raw))
    print()

Считывается data\got1.txt
Сборка длиной 1770659

Считывается data\got2.txt
Сборка длиной 4071041

Считывается data\got3.txt
Сборка длиной 6391405

Считывается data\got4.txt
Сборка длиной 8107945

Считывается data\got5.txt
Сборка длиной 9719485



In [4]:
#Разбиваем строку на предложения. Каждое предложение с новой строки
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sentences = tokenizer.tokenize(corpus_raw)
print('Всего %d предложений' % len(raw_sentences))

Всего 128868 предложений


In [5]:
#Конвертируем предложения в массивы слов, убирая знаки пунктуации
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

In [6]:
#Проверка на пустые строки
sentences = []
for raw_sentence in raw_sentences:
    if(len(raw_sentence) > 0):
        sentences.append(sentence_to_wordlist(raw_sentence))

In [7]:
print('Пример:')
print('Предложение:', raw_sentences[13])
print('Получаем:', sentence_to_wordlist(raw_sentences[13]))

Пример:
Предложение: Now he thought of them as old friends.
Получаем: ['Now', 'he', 'thought', 'of', 'them', 'as', 'old', 'friends']


In [8]:
token_count = sum([len(sentence) for sentence in sentences])
print('Всего %d токенов' % token_count)

Всего 1818103 токенов


In [9]:
#Размер вектора
num_features = 300

#минимальное пороговое значение
min_word_count = 3

#количество потоков
num_workers = multiprocessing.cpu_count() #возвращает к-во ядер процессора

#размер окна
context_size = 7

#Downsample setting for frequent words
downsampling = 1e-3 #Not see too many times the most frequent words...

#Seed for the randomnessss, to make results reproducible
seed = 1

In [10]:
got2Vec = w2v.Word2Vec(sg=1,
                           seed=seed,
                           workers=num_workers,
                           size=num_features,
                           min_count=min_word_count,
                           window=context_size,
                           sample=downsampling
)

In [11]:
got2Vec.build_vocab(sentences)

In [12]:
print('Длина словаря Word2Vec: %d' % got2Vec.corpus_count)

Длина словаря Word2Vec: 128868


In [13]:
#Тренируем модель
got2Vec.train(sentences=sentences,total_examples=got2Vec.corpus_count,epochs=got2Vec.iter)

7022448

In [14]:
#Сохраняемся
got2Vec.save('got2Vec.w2v')

In [15]:
#Близкие слова к слову "Drogo"
got2Vec.most_similar("Drogo")

[('Khal', 0.9183875322341919),
 ('khal', 0.8146138191223145),
 ('khalasar', 0.7380660772323608),
 ('bloodriders', 0.7342690229415894),
 ('Dothraki', 0.7175328731536865),
 ('Mirri', 0.7166121006011963),
 ('Maz', 0.7153828740119934),
 ('womb', 0.7110382914543152),
 ('Duur', 0.7098428010940552),
 ('Rhaego', 0.7060027122497559)]

In [16]:
#Близкое значение к словам 'Арья' и 'Бриенна'
got2Vec.most_similar(positive=['Arya', 'Brienne'])[0][0]

'Catelyn'

In [17]:
#Лишний среди Серсеи, Тириона, Джейме и Дейнерис
got2Vec.doesnt_match('Cersei, Jaime, Tyrion, Daenerys'.split())

'Daenerys'

In [19]:
#Близкое значение к словам 'Дрого' и 'Дайнерис'
got2Vec.most_similar(positive=['Khalee', 'Daenerys'])[0][0]

'khal'